# Spatial Data Science no planejamento e organização da colheita no campo

Que a **Ciência do Onde** é de extrema utilidade no Mundo Agro não há dúvidas. Pergunte a qualquer fazendeiro, e ele te dirá que um bom **mapa** ajuda muito todos os seus trabalhos, se orgulhará de dizer que pensa em vários aspectos espaciais como a distancia percorrida, o tipo do solo de cada localidade, a área total de cada talhão, e óbvio a localização exata de cada um deles. Neste contexto adicionar recursos de decisão baseada em dados para esses profissionais é sempre um grande desafio e um grande diferencial para uma solução.
<br><br>
Na semana passada (no dia 20/08/2020) aconteceu o evento ***AgriSummit 2020*** organizado pela Imagem, no qual apresentamos uma solução que fizemos para ajudar o homem do campo, em especial aqueles que planejam colheitas e gerenciam suas frotas, a planejar e sequenciar melhor a colheita dos talhões e áreas. 
<br><br>
A solução proposta é uma solução completa de *Spatial Data Science*, ela passa pela previsão da época ótima para colheita, através de um ou mais indicadores da planta, no caso para a cana utilizamos o TCH e o ATR, e em seguida sequenciamos cada um dos talhões preditos para aquele ano, em uma sequencia otimizada, e trouxemos todos esses dados para um painel dinâmico que auxilia o planejador em suas decisões.
<br><br>
O painel especialmente, baseado inteiramente na web, permite que o planejador o acesse de qualquer parte do mundo onde ele possua conexão com a internet, e tome suas decisões com confiança e bem informado, sem precisar movimentar-se até o escritório para isso. Além disso, o painel interativo, permite que os times discutam possíveis planos de ação enquanto olham para os mesmos dados de maneira simultânea e confiável.
No entanto para chegar a esta solução alguns passos foram necessários, neste artigo pretendo mostrar alguns deles.

### O primeiro passo 
<br>
O primeiro passo foi importar todos os dados existentes das colheitas anteriores, para que pudéssemos utilizá-los no treinamento de um modelo de aprendizado de máquinas que pudesse gerar as previsões das variáveis de campo para posteriormente determinar o melhor período para a colheita e a sequência de colheita.
<br>
<br>
Todos os dados foram compilados em um único *dataset* (tabela para uso nos modelos), de modo a termos toda a informação que precisássemos a mão para a definição do melhor modelo possível.

In [133]:
import pandas as pd
import arcpy as ar
import tensorflow as tf
import sklearn
import numpy as np

In [134]:
df1=pd.read_csv(r"F:\Users\Home\Documents\AGRO-Analytics\2015.csv", sep =';')
df2=pd.read_csv(r"F:\Users\Home\Documents\AGRO-Analytics\2016.csv", sep =';')
df3=pd.read_csv(r"F:\Users\Home\Documents\AGRO-Analytics\2017.csv", sep =';')
df4=pd.read_csv(r"F:\Users\Home\Documents\AGRO-Analytics\2018.csv", sep =';')
df5=pd.read_csv(r"F:\Users\Home\Documents\AGRO-Analytics\2019.csv", sep =';')

dfall = df1
dfall = dfall.append(df2)
dfall = dfall.append(df3)
dfall = dfall.append(df4)
dfall = dfall.append(df5)
dfall.head()

,ID_PROP_TALH_SAFRA,SAFRA,PROPRIEDADE,DESCRICAOPROPRIEDADE,TALHAO,VARIEDADE,DESCRICAOVARIEDADE,CORTE,AMBIENTE,DESCRICAOTIPOSOLO,AMBIENTE_MANEJO,DT_PLANTIO,NRO_SEMANA_PLANTIO,DT_COLHEITA_SF_ANT,NRO_SEMANA_COLHEITA_SF_ANT,DT_COLHEITA,NRO_SEMANA_COLHEITA,DIAS_ENTRE_PLANTIO_COLH,TCH,ATR
0,200012015.0,2015,2,Sitio Fortaleza,1,104,CTC 4,2,A,Lvef (Latossolo Vermelho Eutro,NaN,41277,1,41834,29,42233,34,956,117.002,146.3994
1,200022015.0,2015,2,Sitio Fortaleza,2,138,IACSP 96-2042,2,A,Lvef (Latossolo Vermelho Eutro,NaN,41277,1,41834,29,42233,34,956,60.155,146.6101
2,200032015.0,2015,2,Sitio Fortaleza,3,104,CTC 4,2,A,Lvef (Latossolo Vermelho Eutro,NaN,41277,1,41835,29,42233,34,956,115.532,145.4573
3,200042015.0,2015,2,Sitio Fortaleza,4,104,CTC 4,2,A,Lvef (Latossolo Vermelho Eutro,NaN,41277,1,41834,29,42233,34,956,71.452,147.6969
4,200052015.0,2015,2,Sitio Fortaleza,5,104,CTC 4,2,A,Lvef (Latossolo Vermelho Eutro,NaN,41277,1,41834,29,42233,34,956,98.259,146.7728


<br>
<br>
Neste passo também olhamos o tipo de dados presente em cada variável, para que possamos saber como tratar cada uma das variáveis antes de inseri-las no modelo.
<br>
<br>

In [135]:
dfall.dtypes

ID_PROP_TALH_SAFRA            float64
SAFRA                           int64
PROPRIEDADE                     int64
DESCRICAOPROPRIEDADE           object
TALHAO                          int64
VARIEDADE                       int64
DESCRICAOVARIEDADE             object
CORTE                           int64
AMBIENTE                       object
DESCRICAOTIPOSOLO              object
AMBIENTE_MANEJO               float64
DT_PLANTIO                      int64
NRO_SEMANA_PLANTIO              int64
DT_COLHEITA_SF_ANT              int64
NRO_SEMANA_COLHEITA_SF_ANT      int64
DT_COLHEITA                     int64
NRO_SEMANA_COLHEITA             int64
DIAS_ENTRE_PLANTIO_COLH         int64
TCH                           float64
ATR                           float64
dtype: object

<h3><font color=blue> O segundo passo </font></h3>
<br>
Para esse caso de uso, especificamente, decidimos seguir nossa aplicação anterior, e gerar um modelo *Random Forest Regressor* para fazer a predição dos indicadores de colheita, para isso defnimos 5 funções:
<br>
<br>
 1. Prepare_data_multi: Função que separa os dados entre treino e teste, para que o modelo seja treinado com uma quantidade adequada de dados, e testado com uma outra porção de dados, para validar seu resultado de treino.
<br>
<br>
 2. RF_multi_dias: Função que faz o treinamento do modelo que fará a previsão dos dias esperados para a colheita.
<br>
<br>
 3. RF_multi_TCH: Função que faz o treinamento do modelo que fará a previsão do TCH esperado para cada área.
<br>
<br>
 4. RF_multi_ATR: Função que faz o treinamento do modelo que fará a previsão do ATR esperado para cada área.
<br>
<br>
 5. test_data: Função para testar os melhores hiperparametros que serão utilizados para cada modelo de previsão.
<br>
<br>
Em todos os casos, ao final do treinamento o modelo é salvo para que usemos ele nos passos seguintes a este, que serão a geração dos dados de planejamento de fato.

In [136]:
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pickle
from joblib import dump, load

def __prepare_data_multi(data, campos, alvo, prazo):
        features_considered = campos
        features = data[features_considered]
        dataset = pd.DataFrame(features)
        x = dataset.drop([alvo], axis=1)
        y = dataset[alvo].values
        tempo = -prazo
        x_train = x
        x_test = x
        y_train = y
        y_test = y
        return x_train, x_test, y_train, y_test

def RF_multi_dias(self, data, fields, alvo, prazo):
        x_train, x_test, y_train, y_test = __prepare_data_multi(data, fields, alvo, prazo)
        #reg = RandomForestRegressor(max_depth= 50, min_samples_leaf= 1, min_samples_split= 2)
        reg = RandomForestRegressor(max_depth= 100, max_features='auto', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 400)
        #reg = RandomForestRegressor()
        reg.fit(x_train, y_train)
        dump(reg, 'dias.joblib') 
        tempo = -prazo
        previsao = reg.predict(x_train)
        previsao5dias = pd.DataFrame(previsao)
        rq_ridge = reg.score(x_train, y_train)
        return previsao5dias, rq_ridge

def RF_multi_TCH(self, data, fields, alvo, prazo):
        x_train, x_test, y_train, y_test = __prepare_data_multi(data, fields, alvo, prazo)
        #reg = RandomForestRegressor(max_depth= 50, min_samples_leaf= 1, min_samples_split= 2)
        reg = RandomForestRegressor(max_depth= 110, max_features= 'auto', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 400)
        reg.fit(x_train, y_train)
        dump(reg, 'tch.joblib') 
        tempo = -prazo
        previsao = reg.predict(x_train)
        previsao5dias = pd.DataFrame(previsao)
        rq_ridge = reg.score(x_train, y_train)
        return previsao5dias, rq_ridge

def RF_multi_ATR(self, data, fields, alvo, prazo):
        x_train, x_test, y_train, y_test = __prepare_data_multi(data, fields, alvo, prazo)
        #reg = RandomForestRegressor(max_depth= 50, min_samples_leaf= 1, min_samples_split= 2)
        reg = RandomForestRegressor(max_depth= 110, max_features= 'auto', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 400)
        reg.fit(x_train, y_train)
        dump(reg, 'atr.joblib')
        tempo = -prazo
        previsao = reg.predict(x_train)
        previsao5dias = pd.DataFrame(previsao)
        rq_ridge = reg.score(x_train, y_train)
        return previsao5dias, rq_ridge

def test_data(self, data, fields, alvo, prazo):
    x_train, x_test, y_train, y_test = __prepare_data_multi(data, fields, alvo, prazo)
    # escolha dos primeiros parametros com random search
    random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [int(x) for x in np.linspace(1, 500, num = 11)],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]}
    rf = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10,cv = 3, verbose=2, random_state=42, n_jobs = -1)
    rf_random.fit(x_train, y_train)
    best_random = rf_random.best_estimator_
    print("Best params: {}".format(rf_random.best_params_))
    #Best params: {'n_estimators': 1200, 'min_samples_split': 2, 'min_samples_leaf': 4,'max_features': 'auto', 'max_depth': 100, 'bootstrap': True}
    #escolha dos parametros finais com o grid search
    param_grid = {
    'bootstrap': [False],
    'max_depth': [40, 50, 90, 100, 110],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'min_samples_split': [2, 4, 6],
    'n_estimators': [50, 100, 200, 400, 600, 800, 1200]
    }
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 3, n_jobs = -1, verbose= 2)
    grid_search.fit(x_train, y_train)
    print("Best params: {}".format(grid_search.best_params_))
    #Best params: {'bootstrap': True, 'max_depth': 90, 'max_features': 'auto', 'min_samples_leaf': 3,'min_samples_split': 10, 'n_estimators': 1200}

<h3><font color=blue> O terceiro passo </font></h3>
<br>
Neste ponto precisamos realmente preparar os dados que utilizaremos, para isso vamos começar com uma verificação da distribuição estatística das variáveis numéricas do modelo. Para que entendamos, quantos dados possuímos, qual a distribuição estatística é provável que eles possuam, e avaliar a presença de valores espúrios nos dados.


In [137]:
dfall.describe()

,ID_PROP_TALH_SAFRA,SAFRA,PROPRIEDADE,TALHAO,VARIEDADE,CORTE,AMBIENTE_MANEJO,DT_PLANTIO,NRO_SEMANA_PLANTIO,DT_COLHEITA_SF_ANT,NRO_SEMANA_COLHEITA_SF_ANT,DT_COLHEITA,NRO_SEMANA_COLHEITA,DIAS_ENTRE_PLANTIO_COLH,TCH,ATR
count,2.212300e+04,22123.000000,22123.000000,22123.000000,22123.000000,22123.000000,0.0,22123.000000,22123.000000,22123.000000,22123.000000,22123.000000,22123.000000,22123.000000,22123.000000,22123.000000
mean,4.863283e+10,2016.847399,486.327126,12.832572,103.473670,3.642544,NaN,41453.758261,20.057904,42494.546400,28.079103,42880.484609,29.236812,1426.726348,92.234606,128.537780
std,4.067744e+10,1.346773,406.774992,15.190303,31.770048,1.878716,NaN,1252.155138,16.995419,494.435836,11.960244,484.683248,8.988572,1182.462471,37.910557,29.140198
min,2.000120e+08,2015.000000,2.000000,1.000000,9.000000,0.000000,NaN,38449.000000,1.000000,41333.000000,1.000000,41862.000000,4.000000,-69917.000000,0.786000,0.000000
25%,1.240027e+10,2016.000000,124.000000,3.000000,103.000000,2.000000,NaN,40955.000000,7.000000,42105.000000,19.000000,42501.000000,21.000000,881.000000,69.755500,120.368150
50%,3.960005e+10,2017.000000,396.000000,7.000000,107.000000,4.000000,NaN,41442.000000,12.000000,42512.000000,28.000000,42904.000000,29.000000,1367.000000,87.789000,132.135200
75%,7.250005e+10,2018.000000,725.000000,16.000000,130.000000,5.000000,NaN,42017.000000,41.000000,42922.000000,38.000000,43292.000000,37.000000,1946.000000,108.895000,144.897800
max,1.376000e+11,2019.000000,1376.000000,129.000000,169.000000,12.000000,NaN,112205.000000,53.000000,43469.000000,53.000000,43705.000000,51.000000,4288.000000,585.782000,265.087600


<br>
Por exemplo no caso da variável DIAS_ENTRE_PLANTIO_COLH que será utilizada para a previsão do número de dias entre o plantio e a colheita e determinar a data provável de colheita, possuía pelo menos um valor negativo (-69917.000000), decorrente possivelmente de algum erro de digitação ou formatação nas planilhas originais, então para nos assegurarmos que faremos as melhores predições, vamos excluir todos registros que possuam nesta coluna os valores menores do que 1 (ou seja zeros e negativos).
<br>
<br>
Podemos ver na célula abaixo que aproximadamente 200 registros foram excluídos da nossa tabela de dados, o que é um valor pequeno (cerca de 1%) dos dados, corroborando a ideia de erros de digitação ou de formatação.
<br>

In [138]:
indexNames = dfall[ (dfall['DIAS_ENTRE_PLANTIO_COLH'] <1)].index
dfall.drop(indexNames , inplace=True)
dfall.describe()

,ID_PROP_TALH_SAFRA,SAFRA,PROPRIEDADE,TALHAO,VARIEDADE,CORTE,AMBIENTE_MANEJO,DT_PLANTIO,NRO_SEMANA_PLANTIO,DT_COLHEITA_SF_ANT,NRO_SEMANA_COLHEITA_SF_ANT,DT_COLHEITA,NRO_SEMANA_COLHEITA,DIAS_ENTRE_PLANTIO_COLH,TCH,ATR
count,2.191200e+04,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,0.0,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000
mean,4.866361e+10,2016.847207,486.634995,12.849489,103.524553,3.638143,NaN,41437.230924,20.063436,42494.383717,28.068364,42880.377236,29.231426,1443.146313,92.279162,128.556082
std,4.077785e+10,1.346643,407.779161,15.196685,31.714673,1.875499,NaN,813.064910,16.989794,494.221794,11.956594,484.502602,8.996253,696.166883,37.942434,29.100548
min,2.000120e+08,2015.000000,2.000000,1.000000,9.000000,0.000000,NaN,38449.000000,1.000000,41333.000000,1.000000,41862.000000,5.000000,73.000000,0.786000,0.000000
25%,1.240024e+10,2016.000000,124.000000,3.000000,103.000000,2.000000,NaN,40955.000000,7.000000,42106.000000,19.000000,42501.000000,21.000000,883.000000,69.812750,120.369350
50%,3.960006e+10,2017.000000,396.000000,7.000000,107.000000,4.000000,NaN,41430.000000,12.000000,42512.000000,28.000000,42904.000000,29.000000,1368.000000,87.837000,132.123800
75%,7.252506e+10,2018.000000,725.250000,16.000000,130.000000,5.000000,NaN,42015.000000,41.000000,42922.000000,38.000000,43291.000000,37.000000,1947.000000,108.988500,144.896900
max,1.376000e+11,2019.000000,1376.000000,129.000000,169.000000,12.000000,NaN,43469.000000,53.000000,43469.000000,53.000000,43705.000000,51.000000,4288.000000,585.782000,265.087600


In [139]:
list(dfall)

['ID_PROP_TALH_SAFRA',
 'SAFRA',
 'PROPRIEDADE',
 'DESCRICAOPROPRIEDADE',
 'TALHAO',
 'VARIEDADE',
 'DESCRICAOVARIEDADE',
 'CORTE',
 'AMBIENTE',
 'DESCRICAOTIPOSOLO',
 'AMBIENTE_MANEJO',
 'DT_PLANTIO',
 'NRO_SEMANA_PLANTIO',
 'DT_COLHEITA_SF_ANT',
 'NRO_SEMANA_COLHEITA_SF_ANT',
 'DT_COLHEITA',
 'NRO_SEMANA_COLHEITA',
 'DIAS_ENTRE_PLANTIO_COLH',
 'TCH',
 'ATR']

In [140]:
dfselect = dfall[['VARIEDADE','CORTE','AMBIENTE','DESCRICAOTIPOSOLO','NRO_SEMANA_PLANTIO', 'DIAS_ENTRE_PLANTIO_COLH', 'ATR', 'TCH']]

Neste ponto, ainda no terceiro passo, separamos as variáveis que utilizaremos (célula acima) e em seguida preparamos o banco de dados que utilizaremos para os modelos.
<br>
<br>
A função 'get_dummies' é utilizada aqui, para transformar variáveis categóricas em colunas de valores binários (0 e 1) que facilitam o trabalho do modelo de regressão, pois este consegue tratar melhor essas variáveis dentro da previsão que farão, ao invés de uma classificação baseada em valores inteiros (categorias de 1 a 5 por exemplo) que poderiam ser interpretadas como variáveis numéricas pelo modelo.
<br>
<br>
Desta forma ficamos com o banco de dados no formato listado abaixo, com várias colunas com nome similar, indicando a coluna de origem e o nome da categoria orginal.No fim ficamos com 29 colunas de dados.


In [141]:
dfselect = pd.get_dummies(dfselect)

In [142]:
colunas= list (dfselect)
colunas

['VARIEDADE',
 'CORTE',
 'NRO_SEMANA_PLANTIO',
 'DIAS_ENTRE_PLANTIO_COLH',
 'ATR',
 'TCH',
 'AMBIENTE_A',
 'AMBIENTE_B',
 'AMBIENTE_C',
 'AMBIENTE_D',
 'DESCRICAOTIPOSOLO_GXd (Gleissolo Háplico Distrof',
 'DESCRICAOTIPOSOLO_LAa ( LATOSSOLO AMARELO Álico ',
 'DESCRICAOTIPOSOLO_LAd (Latossolo Amarelo Distróf',
 'DESCRICAOTIPOSOLO_LAw (LATOSSOLO AMARELO ácrico)',
 'DESCRICAOTIPOSOLO_LRd (Latossolo Roxo Distrofico',
 'DESCRICAOTIPOSOLO_LRe (Latossolo Roxo Eutrofico)',
 'DESCRICAOTIPOSOLO_LVA (Latossolo Vermelho Amarel',
 'DESCRICAOTIPOSOLO_LVAd (Latossolo Vermelho Amare',
 'DESCRICAOTIPOSOLO_LVAe (Latossolo Vermelho Amare',
 'DESCRICAOTIPOSOLO_LVAw (Latossolo Vermelho-Amare',
 'DESCRICAOTIPOSOLO_LVd (Latossolo Vermelho Distro',
 'DESCRICAOTIPOSOLO_LVdf (Latossolo Vermelho Distr',
 'DESCRICAOTIPOSOLO_LVe (latossolo Vermelho Eutrof',
 'DESCRICAOTIPOSOLO_LVw (Latossolo Vermelho Acrico',
 'DESCRICAOTIPOSOLO_LVwf (Latossolo Vermelho Acrif',
 'DESCRICAOTIPOSOLO_Lae (Latossolo Amarelo Eutrofi',
 

In [143]:
dfselect.head()

,VARIEDADE,CORTE,NRO_SEMANA_PLANTIO,DIAS_ENTRE_PLANTIO_COLH,ATR,TCH,AMBIENTE_A,AMBIENTE_B,AMBIENTE_C,AMBIENTE_D,...,DESCRICAOTIPOSOLO_LVAw (Latossolo Vermelho-Amare,DESCRICAOTIPOSOLO_LVd (Latossolo Vermelho Distro,DESCRICAOTIPOSOLO_LVdf (Latossolo Vermelho Distr,DESCRICAOTIPOSOLO_LVe (latossolo Vermelho Eutrof,DESCRICAOTIPOSOLO_LVw (Latossolo Vermelho Acrico,DESCRICAOTIPOSOLO_LVwf (Latossolo Vermelho Acrif,DESCRICAOTIPOSOLO_Lae (Latossolo Amarelo Eutrofi,DESCRICAOTIPOSOLO_Lvef (Latossolo Vermelho Eutro,DESCRICAOTIPOSOLO_NVdf (Nitossolo Vermelho Distr,DESCRICAOTIPOSOLO_NVef (Nitossolo Vermelho Eutro
0,104,2,1,956,146.3994,117.002,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,138,2,1,956,146.6101,60.155,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,104,2,1,956,145.4573,115.532,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,104,2,1,956,147.6969,71.452,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,104,2,1,956,146.7728,98.259,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


<br>
<br>
Por fim, apenas por segurança executamos um processo de 'fillna' que preenche qualquer célula vazia no dataset, com um 0, o que garante o trabalho do modelo, sem perigo de um possível erro por conta de dados vazios.
<br>
<br>

In [144]:
dfselect = dfselect.fillna(0)

<h3><font color=blue> O quarto passo </font></h3>
<br>
Neste passo fazemos os treinos dos modelos e avaliamos todos os dados de erros atribuidos aos modelos.
<br>
<br>
O primeiro modelo a ser avaliado neste caso, é do numero de dias até a colheita, que apresentou uma boa aderencia aos dados (com um R² acima de 0.99, e um erro médio absoluto de 20 dias, que para esse caso é aceitavel, pois para determinar o melhor período num segundo momento plotaremos vários dias próximos a essa data, para determinar o melhor dia de colheita, quando conseguiremos maximizar os dados de ATR e TCH.
<br>
<br>

In [145]:
previsto = RF_multi_dias('self', dfselect, colunas, 'DIAS_ENTRE_PLANTIO_COLH', 365)
previsto

(               0
 0       954.4375
 1       894.0875
 2       953.0400
 3       957.9450
 4       944.3225
 ...          ...
 21907   515.8550
 21908   526.5725
 21909   519.3750
 21910  1201.1175
 21911   895.4075
 
 [21912 rows x 1 columns],
 0.9966130041140306)

In [146]:
y = dfselect[ 'DIAS_ENTRE_PLANTIO_COLH'].values.reshape(-1, 1)
y_test = y

In [147]:
from sklearn.metrics import mean_squared_error,  mean_absolute_error
y_true = y_test
y_pred = previsto[0]
MSE = mean_squared_error(y_true, y_pred)
MAE=  mean_absolute_error(y_true, y_pred)
print("MSE = ", MSE, "MAE=", MAE)

MSE =  1641.426980956211 MAE= 20.5136612358525


<br>
<br>
Para o TCH, fazemos o mesmo processo de treino do modelo, de modo, que possamos avaliar a acertividade dos parametros escolhidos.
<br>
<br>
Neste caso conseguimos um erro médio absoluto de 8 unidades, que representa cerca de 5% do valor do TCH, em um modelo com o R² de 0.89, que é um resultado bastante acima do esperado, e um excelente resultado quando comparado a métricas menos automatizadas que os modelos de machine learning.
<br>
<br>

In [148]:
dfselect1 = dfselect
dfselect1['DIAS_ENTRE_PLANTIO_COLH']=previsto[0]
previsto1 = RF_multi_TCH('self', dfselect1, colunas, 'TCH', 365)
previsto1

(                0
 0      112.048590
 1       74.709125
 2      113.242180
 3       85.458288
 4      103.147395
 ...           ...
 21907  152.886867
 21908  183.399983
 21909  199.815270
 21910  117.550415
 21911   97.597310
 
 [21912 rows x 1 columns],
 0.8925058884021766)

In [149]:
y = dfselect[ 'TCH'].values.reshape(-1, 1)
y_test = y

In [150]:
from sklearn.metrics import mean_squared_error,  mean_absolute_error
y_true = y_test
y_pred = previsto1[0]
MSE = mean_squared_error(y_true, y_pred)
MAE=  mean_absolute_error(y_true, y_pred)
print("MSE = ", MSE, "MAE=", MAE)

MSE =  154.74450327937166 MAE= 8.018975975835158


<br>
<br>
Por último fazemos o mesmo passo para o ATR, este que é sem dúvida o mais sensível dos indicadores para a cana de açucar, e podemos observar que o modelo possui uma capacidade alta de predição, com um R² aproximado de 0.88 e um erro médio absoluto em torno de 5 unidades, que representa aproximadamente 4% do valor médio de ATR apurado dos dados históricos.
<br>
<br>
Esse percentual é importante, pois o menor erro nesta variável, permite uma segurança no processo de escolha da sequencia de colheita, já que o principal indicador para avaliar a viabilidade da colheita de um determinado talhão é justamente o ATR daquele talhão.
<br>
<br>

In [151]:
dfselect1['TCH']=previsto1[0]
previsto2 = RF_multi_ATR('self', dfselect1, colunas, 'ATR', 365)
previsto2

(                0
 0      145.313691
 1      142.349768
 2      145.161314
 3      147.016093
 4      146.461687
 ...           ...
 21907  119.505698
 21908  135.899656
 21909  135.334294
 21910  131.212391
 21911  146.396653
 
 [21912 rows x 1 columns],
 0.8789000129288882)

In [152]:
y = dfselect[ 'ATR'].values.reshape(-1, 1)
y_test = y

In [153]:
from sklearn.metrics import mean_squared_error,  mean_absolute_error
y_true = y_test
y_pred = previsto2[0]
MSE = mean_squared_error(y_true, y_pred)
MAE=  mean_absolute_error(y_true, y_pred)
print("MSE = ", MSE, "MAE=", MAE)

MSE =  102.54786053586965 MAE= 5.093182366737327


<h3><font color=blue> O quinto passo </font></h3>
<br>
Para que possamos avaliar com calma os resultados e suas diversas nuances antes de implementar os modelos no pipeline que levará a construção do Painel dinâmico, fazemos criamos uma tabela com os dados originais e previstos, e salvamos ela em um arquivo de planilha eletrônica para uso posterior.

In [154]:
dfprevisao = dfselect
dfprevisao['Dias_Previsto'] = previsto[0].values
dfprevisao['TCH_previsto'] = previsto1[0].values
dfprevisao['ATR_Previsto'] = previsto2[0].values

In [155]:
list(dfprevisao)

['VARIEDADE',
 'CORTE',
 'NRO_SEMANA_PLANTIO',
 'DIAS_ENTRE_PLANTIO_COLH',
 'ATR',
 'TCH',
 'AMBIENTE_A',
 'AMBIENTE_B',
 'AMBIENTE_C',
 'AMBIENTE_D',
 'DESCRICAOTIPOSOLO_GXd (Gleissolo Háplico Distrof',
 'DESCRICAOTIPOSOLO_LAa ( LATOSSOLO AMARELO Álico ',
 'DESCRICAOTIPOSOLO_LAd (Latossolo Amarelo Distróf',
 'DESCRICAOTIPOSOLO_LAw (LATOSSOLO AMARELO ácrico)',
 'DESCRICAOTIPOSOLO_LRd (Latossolo Roxo Distrofico',
 'DESCRICAOTIPOSOLO_LRe (Latossolo Roxo Eutrofico)',
 'DESCRICAOTIPOSOLO_LVA (Latossolo Vermelho Amarel',
 'DESCRICAOTIPOSOLO_LVAd (Latossolo Vermelho Amare',
 'DESCRICAOTIPOSOLO_LVAe (Latossolo Vermelho Amare',
 'DESCRICAOTIPOSOLO_LVAw (Latossolo Vermelho-Amare',
 'DESCRICAOTIPOSOLO_LVd (Latossolo Vermelho Distro',
 'DESCRICAOTIPOSOLO_LVdf (Latossolo Vermelho Distr',
 'DESCRICAOTIPOSOLO_LVe (latossolo Vermelho Eutrof',
 'DESCRICAOTIPOSOLO_LVw (Latossolo Vermelho Acrico',
 'DESCRICAOTIPOSOLO_LVwf (Latossolo Vermelho Acrif',
 'DESCRICAOTIPOSOLO_Lae (Latossolo Amarelo Eutrofi',
 

In [156]:
dfprevisao.describe()

,VARIEDADE,CORTE,NRO_SEMANA_PLANTIO,DIAS_ENTRE_PLANTIO_COLH,ATR,TCH,AMBIENTE_A,AMBIENTE_B,AMBIENTE_C,AMBIENTE_D,...,DESCRICAOTIPOSOLO_LVe (latossolo Vermelho Eutrof,DESCRICAOTIPOSOLO_LVw (Latossolo Vermelho Acrico,DESCRICAOTIPOSOLO_LVwf (Latossolo Vermelho Acrif,DESCRICAOTIPOSOLO_Lae (Latossolo Amarelo Eutrofi,DESCRICAOTIPOSOLO_Lvef (Latossolo Vermelho Eutro,DESCRICAOTIPOSOLO_NVdf (Nitossolo Vermelho Distr,DESCRICAOTIPOSOLO_NVef (Nitossolo Vermelho Eutro,Dias_Previsto,TCH_previsto,ATR_Previsto
count,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,...,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000,21912.000000
mean,103.524553,3.638143,20.063436,1381.844523,128.556082,88.710940,0.302483,0.427756,0.209064,0.060697,...,0.016886,0.011227,0.182959,0.000456,0.252464,0.002099,0.033589,1442.907398,92.561916,128.289344
std,31.714673,1.875499,16.989794,685.503623,29.100548,28.437813,0.459344,0.494765,0.406649,0.238780,...,0.128846,0.105362,0.386642,0.021358,0.434436,0.045771,0.180173,690.858685,29.233927,20.984489
min,9.000000,0.000000,1.000000,257.510000,0.000000,16.929222,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,164.697500,11.485260,8.934771
25%,103.000000,2.000000,7.000000,830.085000,120.369350,71.341640,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,881.250625,74.147899,120.346730
50%,107.000000,4.000000,12.000000,1274.610000,132.123800,85.022365,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1373.747500,88.511239,130.347643
75%,130.000000,5.000000,41.000000,1909.837500,144.896900,100.756128,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1940.333750,106.454593,141.590655
max,169.000000,12.000000,53.000000,3716.695000,265.087600,424.888683,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4230.702500,424.888683,217.398443


In [157]:
dfprevisao.to_csv(r"F:\Users\Home\Documents\AGRO-Analytics\DadosEstudoImagemResultados.csv", sep =',')

<h3><font color=blue> Conclusão </font></h3>
<br>
Neste artigo mostramos um pouco do que está por trás da criação do painel dinamico que foi apresentado no AgriSummit 2020, e como utilizamos modelos de regressão por machine learning para que os dados que serão utilizados pelos planejadores das colheitas tenham o maior nível de acurácia possível. 
<br>
<br>
Este no entanto é apenas o primeiro dos passos para a geração do painel, que ainda envolve, a previsão dos valores do ATR e TCH para vários dias proximos da data apontada pelo modelo, para determinar exatamente o dia ótimo da colheita, e então inserir esses dados, para um modelo de roterização que determina o melhor sequenciamento dos talhões pensando no menor deslocamento possível entre a colheita de um talhão e outro, para minimizar assim o principal ponto de atenção que levou à construção deste modelo em um primeiro modelo, que é justamente a economia nos custos de transporte e movimentação de equipamentos, que por sua vez diminui o custo de produção do açucar e traz maior lucratividade às usinas.
<br>
<br>
Também é importante dizer que esses modelos poderiam ser aplicados e construídos pensando em outras culturas e para negócios completamente diferentes dentro do Mundo Agro e fora dele. Afinal com o processo de seleção adequada de váriaveis o mesmo pipeline de processamento e teste de modelos poderia ser aplicado facilmente a outros datasets e modelos.